In [ ]:
import torch import torch.nn as nn import torch.optim as optim from torch.utils.data import DataLoader from torchvision import datasets, transforms, models from torchvision.models import efficientnet_b4, EfficientNet_B4_Weights from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score import matplotlib.pyplot as plt import seaborn as sns import numpy as np # Hyperparameters batch_size = 32 learning_rate = 0.0001 num_epochs = 25 weight_decay = 1e-4 # L2 regularization dropout_rate = 0.6 num_classes = 2 # For your 'fake' and 'real' classes patience = 3 # Number of epochs to wait for improvement before stopping # Data augmentation and normalization
train_transforms = transforms.Compose([ transforms.RandomResizedCrop(224), transforms.RandomHorizontalFlip(), transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1), transforms.RandomRotation(10), transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) ]) val_transforms = transforms.Compose([ transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) ]) # Load datasets train_dataset = datasets.ImageFolder(root='/kaggle/input/audiospec/output_spectrograms_training/output_spectrograms_training', transform=train_transforms) val_dataset = datasets.ImageFolder(root='/kaggle/input/audiospec/output_spectrograms_training/output_spectrograms_training', transform=val_transforms) # DataLoaders train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True) val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True) # Load EfficientNet-B4 model model = efficientnet_b4(weights=EfficientNet_B4_Weights.DEFAULT) num_features = model.classifier[1].in_features model.classifier = nn.Sequential( nn.Dropout(dropout_rate), nn.Linear(num_features, num_classes) ) device = torch.device("cuda" if torch.cuda.is_available() else "cpu") model = model.to(device) # Loss function and optimizer
criterion = nn.CrossEntropyLoss() optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay) # Early Stopping class class EarlyStopping: def __init__(self, patience=3, verbose=False): self.patience = patience self.verbose = verbose self.counter = 0 self.best_loss = float('inf') self.early_stop = False def __call__(self, val_loss): if val_loss < self.best_loss: self.best_loss = val_loss self.counter = 0 else: self.counter += 1 if self.counter >= self.patience: self.early_stop = True if self.verbose: print(f'Early stopping triggered after {self.patience} epochs without improvement.') # Track metrics train_losses, val_losses = [], [] train_accuracies, val_accuracies = [], [] # Define a learning rate scheduler scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5) # Training and validation loop with learning rate scheduler def train_and_evaluate(model, criterion, optimizer, num_epochs): early_stopping = EarlyStopping(patience=patience, verbose=True) for epoch in range(num_epochs): model.train() running_loss = 0.0 correct = 0 total = 0 # Training loop for images, labels in train_loader: images, labels = images.to(device), labels.to(device)
outputs = model(images) loss = criterion(outputs, labels) optimizer.zero_grad() loss.backward() optimizer.step() running_loss += loss.item() _, predicted = torch.max(outputs.data, 1) total += labels.size(0) correct += (predicted == labels).sum().item() train_accuracy = 100 * correct / total train_loss = running_loss / len(train_loader) train_losses.append(train_loss) train_accuracies.append(train_accuracy) # Validation loop model.eval() val_loss = 0.0 val_correct = 0 val_total = 0 all_labels = [] all_predictions = [] with torch.no_grad(): for images, labels in val_loader: images, labels = images.to(device), labels.to(device) outputs = model(images) loss = criterion(outputs, labels) val_loss += loss.item() _, predicted = torch.max(outputs.data, 1) val_total += labels.size(0) val_correct += (predicted == labels).sum().item() # Store labels and predictions for metric calculations all_labels.extend(labels.cpu().numpy()) all_predictions.extend(predicted.cpu().numpy())
val_accuracy = 100 * val_correct / val_total val_loss = val_loss / len(val_loader) val_losses.append(val_loss) val_accuracies.append(val_accuracy) # Calculate precision, recall, and F1 score precision = precision_score(all_labels, all_predictions, average='macro') recall = recall_score(all_labels, all_predictions, average='macro') f1 = f1_score(all_labels, all_predictions, average='macro') # Step the scheduler scheduler.step() print(f'Epoch [{epoch+1}/{num_epochs}], ' f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, ' f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%, ' f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}') # Check for early stopping early_stopping(val_loss) if early_stopping.early_stop: print("Early stopping triggered.") break # Save the model weights torch.save(model.state_dict(), 'efficientnet_b4_model_weights.pth') # Function to plot training and validation metrics def plot_training_progress(): epochs = range(1, len(train_losses) + 1) plt.figure(figsize=(12, 5)) # Plot loss plt.subplot(1, 2, 1) plt.plot(epochs, train_losses, label='Train Loss') plt.plot(epochs, val_losses, label='Validation Loss') plt.xlabel('Epochs') plt.ylabel('Loss') plt.title('Training and Validation Loss') plt.legend()
# Plot accuracy plt.subplot(1, 2, 2) plt.plot(epochs, train_accuracies, label='Train Accuracy') plt.plot(epochs, val_accuracies, label='Validation Accuracy') plt.xlabel('Epochs') plt.ylabel('Accuracy') plt.title('Training and Validation Accuracy') plt.legend() plt.show() # Function to generate and display the confusion matrix def display_confusion_matrix(data_loader, dataset_type='Training'): all_labels = [] all_predictions = [] model.eval() with torch.no_grad(): for images, labels in data_loader: images, labels = images.to(device), labels.to(device) outputs = model(images) _, predicted = torch.max(outputs.data, 1) all_labels.extend(labels.cpu().numpy()) all_predictions.extend(predicted.cpu().numpy()) cm = confusion_matrix(all_labels, all_predictions) plt.figure(figsize=(8, 6)) sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=train_dataset.classes, yticklabels=train_dataset.classes) plt.xlabel('Predicted') plt.ylabel('Actual') plt.title(f'Confusion Matrix - {dataset_type}') plt.show() # Run the training and validation train_and_evaluate(model, criterion, optimizer, num_epochs) # Plot the training and validation metrics plot_training_progress() # Display confusion matrices for training and validation sets
display_confusion_matrix(train_loader, 'Training') display_confusion_matrix(val_loader, 'Validation')